## Notebook Covers following topics:
- **Built-in scope**
- **Global scope**
- **Local scope**
- **Non-local scope**
- **'global' keyword**
- **'nonlocal' keyword**
- **Closures**
    - How to check if a function is closure or not using **'fn.__ closure__'**, fn -> function name
- **Free Variables**
    - How to check free variables using **'fn.__ code__ .co_freevars'**, fn -> function name
- **How closures can replace classes. Explained with 'Averager' example**
- **How to call a object directly using __ call__ method (like fwd in Pytorch)**
- **Closure example to update a counter as & when a function is called**

In [158]:
#Imports required for this notebook
from time import perf_counter

In [1]:
def fn():
    x = 2 # x is defined inside fn & hence has local scope
    print(f'Inside fn, x is {x}')
fn()

Inside fn, x is 2


In [2]:
print(x)  # will fail as now we are on global scope

NameError: name 'x' is not defined

In [3]:
def fn():
    global x  #we are defining global scope for x, so will work anywhere within module (ie .py file)
    x = 10
    print(f'Inside fn, x is {x}')
fn()

Inside fn, x is 10


In [4]:
print(x)

10


In [7]:
a = 10

def my_func():
    global a
    a = 'hello'
    print('global a:', a)
print(a)  # This will print global scope before calling function i.e. 10
my_func() # This will print value assigned inside my_func i.e. 'hello'
print(a)  # 'a' is now globally 'hello'. so will print 'hello' 

10
global a: hello
hello


In [9]:
a = 10
f = lambda n: print(a ** n)  # This will work as 'a' is defined before
f(2)

100


In [10]:
f = lambda n: print(b ** n) # This will fail as b is not there
f(2)

NameError: name 'b' is not defined

In [11]:
f = lambda b, n: print(b ** n) # We are supplying 'b'
f(3, 2)

9


In [13]:
print(True)  # These are built-in scope, will work anywhere

True


In [17]:
def outer_func():
    x = 'hello'
    def inner_func():
        x = 'red'
        print(f' x1 {x}') # will print inner_func() scope  
    inner_func()
    print(f' x2 {x}') # will print outer_func() scope

outer_func()

 x1 red
 x2 hello


In [18]:
z = 100
def outer_func():
    global z
    print(z)
    def inner_func():
        global z
        print(z)
    inner_func()

outer_func()

100
100


In [20]:
def outer_func():
    x = 'hello'
    def inner():
        x = 'tsai'
        print('inner x:', x)
    inner()
    print('outer x:', x)
    
outer_func()

inner x: tsai
outer x: hello


In [44]:
def outer():
    x = 'hello'
    def inner1():
        def inner2():
            nonlocal x
            x = 'python'
        inner2()
        print(f'defined inner2 {x}')
    inner1()
    print(f'defined inner1 {x}')

outer()

defined inner2 python
defined inner1 python


In [45]:
print(x)

NameError: name 'x' is not defined

***x = 'inner2' below is useless outside inner2(). We can never use the reassigned value outside inner2()***

In [57]:
def outer():
    x = 'hello'
    def inner1():
        x = 'inner1'
        def inner2():
            x = 'inner2'
            print(f'Inside inner2 x = {x}')
        inner2()
        print(f'defined inner2 x = {x}')
    inner1()
    print(f'defined inner1 x = {x}')
outer()

Inside inner2 x = inner2
defined inner2 x = inner1
defined inner1 x = hello


***But with nonlocal we can do that as below***

In [48]:
def outer():
    x = 'hello'
    def inner1():
        nonlocal x
        x = 'inner1'
        def inner2():
            nonlocal x
            x = 'inner2'
        inner2()
        print(f'defined inner2 x = {x}')
    inner1()
    print(f'defined inner1 x = {x}')
outer()

defined inner2 x = inner2
defined inner1 x = inner2


In [50]:
x = 'global'
def outer():
    global x
    x = 'outer'
    def inner():
        nonlocal x  # This will fail because x is already made global
        x = 'inner'
    inner()
    print(x)

SyntaxError: no binding for nonlocal 'x' found (<ipython-input-50-3b300f32413d>, line 6)

In [51]:
del x
def outer():
    global x
    x = 'outer'
    def inner():
        global x 
        x = 'inner'
    inner()
    print(f'defined inner : x ={x}')
outer()
print(f'After calling outer: x ={x}')

defined inner : x =inner
After calling outer: x =inner


In [58]:
x = 0
def fn1():
    def fn2():
        def fn3():
            nonlocal x  # Failed as x is global
            print(x)            
        fn3()
        print('defined fn3')
    fn2()
    print('defined fn2')
fn1()

SyntaxError: no binding for nonlocal 'x' found (<ipython-input-58-68ecf73549fb>, line 5)

***Here we are getting 20 printed outside fn3 only because on 'nonlocal x' statement***

In [68]:
def fn1():
    x = 10
    def fn2():
        
        def fn3():
            nonlocal x
            x =20 
            print(x)            
        fn3()
        print(f'defined fn3 - x {x}')
    fn2()
    print(f'defined fn2  - x {x}')
fn1()

20
defined fn3 - x 20
defined fn2  - x 20


***Without 'nonlocal x', it will take the assignment of 'x' outside fn3 which in this case is x = 10***

In [69]:
def fn1():
    x = 10
    def fn2():
        
        def fn3():
            x =20
            print(x)            
        fn3()
        print(f'defined fn3 - x {x}')
    fn2()
    print(f'defined fn2 - x {x}')
fn1()

20
defined fn3 - x 10
defined fn2 - x 10


## Closures

In [1]:
def outer():
    x = 'python' # free variable
    def inner():
        print(x)
    return inner

In [2]:
fn = outer()

In [3]:
fn()

python


In [4]:
fn.__code__.co_freevars

('x',)

In [5]:
fn.__closure__

(<cell at 0x00000225EB0664F8: str object at 0x00000225E920FEB0>,)

***Below is NOT a closure. Because we are assigning 'x' making it a local scope. Also we didnt give 'nonlocal x'***

In [29]:
def outer():
    x = 123
    print('Outer:', hex(id(x)))
    def inner():
        x = 123                     #NOT a free variable because of this
        print('inner:', hex(id(x)))
    return inner

In [30]:
func = outer()

Outer: 0x7ffcf9a2b0d0


In [31]:
func()

inner: 0x7ffcf9a2b0d0


In [32]:
func.__code__.co_freevars

()

***Below is a closure. Because we are assigning 'x' to y, thus making 'x' a free variable.*** 

In [33]:
def outer():
    x = 'tsai'  
    print('Outer:',hex(id(x)))
    def inner():
        y = x              # 'x' is free variables
        print('inner:',hex(id(y)))
    return inner
fn = outer()

Outer: 0x225eb0d84f0


In [36]:
func = outer()

Outer: 0x225eb0d84f0


In [37]:
func()

inner: 0x225eb0d84f0


In [39]:
func.__code__.co_freevars

('x',)

In [40]:
func.__closure__

(<cell at 0x00000225EB066C18: str object at 0x00000225EB0D84F0>,)

***This is a closure becoz we have made 'count' nonlocal. This enables us to assign/manipulate 'count' without losing its free variable status. We can see how closure memory changes as 'count' increases.***

In [48]:
def outer():
    count = 0
    def inner():
        nonlocal count
        count += 1           
        return count      # free variable
    return inner

fn = outer()
print(fn.__code__.co_freevars)
print(fn.__closure__)

('count',)
(<cell at 0x00000225EB066108: int object at 0x00007FFCF9A2A170>,)


In [49]:
hex(id(0))

'0x7ffcf9a2a170'

In [50]:
fn()
print(fn.__closure__)
print(hex(id(1)))

(<cell at 0x00000225EB066108: int object at 0x00007FFCF9A2A190>,)
0x7ffcf9a2a190


In [51]:
print(fn())
print(fn.__closure__)
print(hex(id(2)))

2
(<cell at 0x00000225EB066108: int object at 0x00007FFCF9A2A1B0>,)
0x7ffcf9a2a1b0


***Shared Closures. We can count increasing when we call either of fn1() or fn2(). We can also see that both fn1 and fn2 shares same memory address***

In [52]:
def outer():
    count = 0
    def inc1():
        nonlocal count
        count += 1
        return count

    def inc2():
        nonlocal count
        count += 1
        return count
    return inc1, inc2
fn1, fn2 = outer()

In [53]:
fn1.__closure__, fn2.__closure__

((<cell at 0x00000225EB066C48: int object at 0x00007FFCF9A2A170>,),
 (<cell at 0x00000225EB066C48: int object at 0x00007FFCF9A2A170>,))

In [54]:
print(fn1())
fn1.__closure__, fn2.__closure__

1


((<cell at 0x00000225EB066C48: int object at 0x00007FFCF9A2A190>,),
 (<cell at 0x00000225EB066C48: int object at 0x00007FFCF9A2A190>,))

In [55]:
print(fn1())
fn1.__closure__, fn2.__closure__

2


((<cell at 0x00000225EB066C48: int object at 0x00007FFCF9A2A1B0>,),
 (<cell at 0x00000225EB066C48: int object at 0x00007FFCF9A2A1B0>,))

In [56]:
print(fn2())
fn1.__closure__, fn2.__closure__

3


((<cell at 0x00000225EB066C48: int object at 0x00007FFCF9A2A1D0>,),
 (<cell at 0x00000225EB066C48: int object at 0x00007FFCF9A2A1D0>,))

In [57]:
fn1.__code__.co_freevars, fn2.__code__.co_freevars

(('count',), ('count',))

***Defining various powers using closures***

In [58]:
def pow(n):
    def inner(x):
        return x ** n  # n is the freevariable
    return inner

In [59]:
square = pow(2)

In [61]:
square.__code__.co_freevars

('n',)

In [62]:
square.__closure__

(<cell at 0x00000225EB066828: int object at 0x00007FFCF9A2A1B0>,)

In [65]:
hex(id(2))

'0x7ffcf9a2a1b0'

In [63]:
square(9)

81

In [66]:
square.__closure__    # This will remain same because 'square' is closure with n =2 

(<cell at 0x00000225EB066828: int object at 0x00007FFCF9A2A1B0>,)

In [68]:
cube = pow(3)
print(cube.__code__.co_freevars)
cube.__closure__

('n',)


(<cell at 0x00000225EB066918: int object at 0x00007FFCF9A2A1D0>,)

In [69]:
hex(id(3))

'0x7ffcf9a2a1d0'

In [71]:
cube(3)

27

In [72]:
cube.__closure__

(<cell at 0x00000225EB066918: int object at 0x00007FFCF9A2A1D0>,)

***Adds a given number to a fixed number using closure, similar to power we defined above***

In [74]:
def adder(n):
    def inner(x):
        return x + n
    return inner

In [75]:
add_1 = adder(1) # Will add 1 to given number
add_2 = adder(2) # add 2
add_3 = adder(3) # add 3

In [76]:
print(hex(id(1)), hex(id(2)), hex(id(3)))
add_1.__closure__, add_2.__closure__, add_3.__closure__

0x7ffcf9a2a190 0x7ffcf9a2a1b0 0x7ffcf9a2a1d0


((<cell at 0x00000225EB066408: int object at 0x00007FFCF9A2A190>,),
 (<cell at 0x00000225EB066498: int object at 0x00007FFCF9A2A1B0>,),
 (<cell at 0x00000225EB066A08: int object at 0x00007FFCF9A2A1D0>,))

In [77]:
add_1(10)

11

In [78]:
add_1(20)

21

In [79]:
add_2(20)

22

***We can't substitute above closure with below one. Bcoz. all three lambda will get stored with last value of 'n' which in this case will be 3***

In [81]:
adders = []

for n in range(1, 4):
    adders.append(lambda x: x + n)

In [85]:
adders

[<function __main__.<lambda>(x)>,
 <function __main__.<lambda>(x)>,
 <function __main__.<lambda>(x)>]

In [87]:
adders[0](10) # This was supposed to give 11 (10+1) but gives 13 bcoz of above said reason

13

In [90]:
adders[1](10) # This was supposed to give 12 (10+2) but gives 13 bcoz of above said reason

13

In [91]:
adders[2](10) # This was supposed to give 13 (10+3) & gives 13 bcoz n got saved as 3 as it was last instance

13

***In Below case, we can creat a closure with 'n' as a free variable. But 'n' again will be stored with last iteration value ie 3 in below case***

In [92]:
def create_adders():
    adders = []
    for n in range(1, 4):
        adders.append(lambda x: x + n)
    return adders

In [96]:
fn_lst = create_adders()

In [98]:
fn_lst[0].__closure__, fn_lst[1].__closure__, fn_lst[2].__closure__ 

((<cell at 0x00000225EB066F18: int object at 0x00007FFCF9A2A1D0>,),
 (<cell at 0x00000225EB066F18: int object at 0x00007FFCF9A2A1D0>,),
 (<cell at 0x00000225EB066F18: int object at 0x00007FFCF9A2A1D0>,))

In [102]:
hex(id(3))

'0x7ffcf9a2a1d0'

In [101]:
fn_lst[0].__code__.co_freevars, fn_lst[1].__code__.co_freevars, fn_lst[2].__code__.co_freevars

(('n',), ('n',), ('n',))

In [105]:
print(fn_lst[0](10), fn_lst[0](10), fn_lst[0](10))  ##We continue to get 13, 13, 13 instead of 11(10+1), 12(10+2), 13(10+3)

13 13 13


***Problem can be solved by introducing 'y' as below. But this is NOT a closure***

In [124]:
del create_adders

In [132]:
def create_adders():
    adders = []
    for n in range(1, 4):
        adders.append(lambda x, y=n: x + y)
    return adders

In [133]:
adders = create_adders()

In [134]:
adders[0](10), adders[1](10), adders[2](10)

(11, 12, 13)

### Closures can replace classes as shown below

***Let us see how class takes the average of numbers that keeps getting added***

In [137]:
class Averager:
    def __init__(self):
        self.numbers = []

    def add(self, number):
        self.numbers.append(number)
        total = sum(self.numbers)
        count = len(self.numbers)
        return total/count

In [138]:
a = Averager()

In [139]:
a.add(10)

10.0

In [140]:
a.add(20)
a.add(30)

20.0

***Now same thing can be achieved via closures. Also, closures are memory efficient than classes***

In [152]:
def averager():
    numbers = []
    def add(number):
        numbers.append(number)  
        total = sum(numbers)    #numbers is free variable
        count = len(numbers) 
        return total/count
    return add

In [144]:
a = averager()
a.__code__.co_freevars

('numbers',)

In [145]:
a(10)
a(20)
a(30)

20.0

In [146]:
b = averager()

In [147]:
b(20)
b(30)

25.0

In [148]:
a.__closure__, b.__closure__

((<cell at 0x00000225EB066978: list object at 0x00000225EAFB8308>,),
 (<cell at 0x00000225EB0666D8: list object at 0x00000225EB12B388>,))

***Same closure can be rewritten with more number of free variable***

In [153]:
def averager():
    total = 0
    count = 0
    def add(number):
        nonlocal total             # free var
        nonlocal count            # free var
        total = total + number    
        count = count + 1
        return total/count
    return add

In [154]:
a = averager()

In [155]:
a.__code__.co_freevars

('count', 'total')

In [156]:
a.__closure__

(<cell at 0x00000225EB066E58: int object at 0x00007FFCF9A2A170>,
 <cell at 0x00000225EB0667C8: int object at 0x00007FFCF9A2A170>)

In [157]:
a(10)
a(20)
a(30)

20.0

### Closure example for timer & counter

In [159]:
from time import perf_counter

In [160]:
perf_counter()

17618.2748475

In [161]:
perf_counter()

17623.1783081

In [162]:
class Timer:
    def __init__(self):
        self.start = perf_counter()

    def poll(self):
        return perf_counter() - self.start

In [163]:
t = Timer()

In [164]:
t.poll()

8.738801200001035

In [165]:
t.poll()

20.829955000001064

***We can get-rid of t.poll() and simply call t() as we do in pytorch by defining a __call__ method***

In [166]:
class Timer:
    def __init__(self):
        self.start = perf_counter()

    def __call__(self):
        return perf_counter() - self.start

In [167]:
t = Timer()

In [168]:
t()

2.67558629999985

In [169]:
t()

4.681163799999922

***Alternative using closure***

In [182]:
def timer():
    start = perf_counter()
    def time_took():
        return perf_counter() - start
    return time_took

In [183]:
t = timer()

In [184]:
t.__closure__

(<cell at 0x00000225EB13FB88: float object at 0x00000225EB167170>,)

In [185]:
t.__code__.co_freevars

('start',)

In [186]:
t()

1.7124652999991667

In [187]:
t()

5.033306999997876

***Counter example***

In [193]:
def counter():
    count = 0
    def inc_count(increment=1):
        nonlocal count
        count += increment
        return count
    return inc_count

In [194]:
ctr = counter()

In [195]:
ctr(0)

0

In [196]:
ctr(1)

1

In [197]:
ctr(10)

11

***Closure example that counts number of times a function is called***

In [227]:
def ctr(fn):
    cnt = 0
    def keep_cnt(*args, **kwargs):
        nonlocal cnt
        cnt += 1
        print(f'function {fn.__name__} called  {cnt} times')
        return fn(*args, **kwargs)
    return keep_cnt

In [228]:
def add(valu1, valu2):
    return valu1 + valu2

In [229]:
ctr_add = ctr(add)

In [230]:
ctr_add.__closure__

(<cell at 0x00000225EB1669D8: int object at 0x00007FFCF9A2A170>,
 <cell at 0x00000225EB166918: function object at 0x00000225EB1605E8>)

In [231]:
ctr_add(3,4)

function add called  1 times


7

In [232]:
def mul(valu1, valu2):
    return valu1 * valu2

In [233]:
ctr_mul = ctr(mul)

In [234]:
ctr_mul(3,4)

function mul called  1 times


12

In [235]:
ctr_mul(10, 20)

function mul called  2 times


200

In [236]:
ctr_add(100, 200)

function add called  2 times


300

***Closure example that counts number of times a function is called. Counter is stored in a dictionary***

In [247]:
def ctr(fn):
    fn_dict = {'add':0, 'mul': 0}
    def update(*args, **kwargs):
        nonlocal fn_dict
        fn_dict[fn.__name__] += 1
        print(f' {fn.__name__} called {fn_dict[fn.__name__]} times')
        return fn(*args, **kwargs)
    return update

In [248]:
ctr_add = ctr(add)
ctr_mul = ctr(mul)
ctr_add.__closure__, ctr_mul.__closure__, ctr_add.__code__.co_freevars, ctr_mul.__code__.co_freevars

((<cell at 0x00000225EB166528: function object at 0x00000225EB1605E8>,
  <cell at 0x00000225EB166AF8: dict object at 0x00000225EB16F908>),
 (<cell at 0x00000225EB166138: function object at 0x00000225EB127EE8>,
  <cell at 0x00000225EB166C18: dict object at 0x00000225EB16FD18>),
 ('fn', 'fn_dict'),
 ('fn', 'fn_dict'))

In [250]:
ctr_add(2,3)
ctr_add(2,3)
ctr_add(2,3)

 add called 2 times
 add called 3 times
 add called 4 times


5

In [251]:
ctr_mul(2,3)
ctr_mul(2,3)
ctr_mul(2,3)

 mul called 1 times
 mul called 2 times
 mul called 3 times


6